<a href="https://colab.research.google.com/github/rafpank/Colab/blob/main/data_science_bootcamp%20/uczenie_maszynowe/03_metryki_regresja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Model regresyjny to taki który ma przewidzieć ciągłą wartość zmiennej czyli np cena akcji na giełdzie, albo liczbę klientów w danym miesiącu/dniu itp

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [7]:
y_true = 100 + 20 * np.random.randn(50) #wygenerowanie 50 wartości z rozkładu normalnego z zakresem do 100 ale tez z odchyleniem 20
y_true

array([119.07195665, 125.06714093, 135.79077791,  71.714106  ,
        94.86076293,  69.80187682, 128.00253117,  98.08097938,
        91.95782372,  78.9142486 ,  88.90361896, 106.79095025,
       110.71928127,  79.99366183, 144.4796644 , 136.08672465,
        89.88746237, 130.8002947 , 102.41673874,  89.98520143,
        84.48972174, 104.65139439,  99.93621033, 102.73126233,
        78.85382594, 116.09857952,  76.77794519, 121.90430144,
        86.78974192,  85.99461517, 125.87593748,  59.16984416,
        99.82596266, 140.3220533 ,  68.07966504,  97.15022884,
        95.1243161 , 105.20805323,  83.33053396, 110.3748254 ,
       103.33215027,  89.09973889, 124.10313429,  98.94634444,
       116.71572672,  91.89043083, 112.62607238, 143.15865904,
       120.20037753, 101.82152982])

In [8]:
y_pred = y_true + 10 * np.random.randn(50)
y_pred

array([123.5104518 , 132.52497089, 142.67681272,  73.81252178,
        94.39815326,  69.38641254, 124.11970666, 108.3833526 ,
        89.2363008 ,  69.14423562,  80.99797338, 111.41115876,
       111.37717145,  66.23047428, 150.26817888, 128.83260143,
        95.68772429, 135.55148873, 102.92075288,  96.41540357,
        88.16389695, 105.65793042, 105.30757175,  95.3638368 ,
        73.95007028, 144.21023936,  64.27810121, 116.55927583,
        90.41043443,  85.31776429, 138.99014775,  54.99605654,
        80.71236574, 131.3685621 ,  75.82631057,  98.81399581,
        89.01267127, 127.71573943,  73.62205176, 109.77081847,
        87.65339745,  82.83219016, 137.4341114 ,  98.75820461,
       109.7310686 ,  87.39129536, 125.96562583, 150.49252229,
       129.62482192,  87.90398673])

In [9]:
result = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})
result.head(10)

,y_true,y_pred
0,119.071957,123.510452
1,125.067141,132.524971
2,135.790778,142.676813
3,71.714106,73.812522
4,94.860763,94.398153
5,69.801877,69.386413
6,128.002531,124.119707
7,98.080979,108.383353
8,91.957824,89.236301
9,78.914249,69.144236


In [10]:
result['error'] = result['y_true'] - result['y_pred']
result.head(10)

,y_true,y_pred,error
0,119.071957,123.510452,-4.438495
1,125.067141,132.524971,-7.457830
2,135.790778,142.676813,-6.886035
3,71.714106,73.812522,-2.098416
4,94.860763,94.398153,0.462610
5,69.801877,69.386413,0.415464
6,128.002531,124.119707,3.882825
7,98.080979,108.383353,-10.302373
8,91.957824,89.236301,2.721523
9,78.914249,69.144236,9.770013


#Interpretacja graficzna


In [13]:
def plot_regression_results(y_true, y_pred):
    results = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred}) #standarowo do wykresu obiekt DataFrame
    min = results[['y_true', 'y_pred']].min().min() #wyznaczam pkt minimum żeby móc wyznaczyć linie trendu
    max = results[['y_true', 'y_pred']].max().max() #wyznaczam pkt maksimum

    fig = go.Figure(data=[go.Scatter(x=results['y_true'], y=results['y_pred'], mode='markers'), #przekazuje wartości do obiektu fig
                    go.Scatter(x=[min, max], y=[min, max])], #przekazujemy drugi ślad (tą czerwoną linię)
                    layout=go.Layout(showlegend=False, width=800, height=500,
                                     xaxis_title='y_true',
                                     yaxis_title='y_pred',
                                     title='Regression results'))
    fig.show()
plot_regression_results(y_true, y_pred)

In [14]:
y_true = 100 + 20 * np.random.randn(1000)
y_pred = y_true + 10 * np.random.randn(1000)
results = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})
results['error'] = results['y_true'] - results['y_pred']

px.histogram(results, x='error', nbins=50, width=800, height=500, title='Error distribution')

Rozkład powinnien być jak najbliżej rozkładu normalnego
jeżeli jest więcej po stronie (-) to oznacza że model przeszacowuje wartości

jeżeli błąd jest bardziej (+) to oznacza że nasz model częściej niedoszacowuje wartości

Najlepiej jest gdy model jest skupiony blisko 0 bo to oznacza, że błędy są dość małe. Oczywiście, ważne jest żeby model nie był zbyt dopasowany do naszych danych, bo to oznacza przeuczenie.


### <a name='a3'></a> Mean Absolute Error - Średni błąd bezwzględny
### $$MAE = \frac{1}{n}\sum_{i=1}^{n}|y_{true} - y_{pred}|$$

In [15]:
def mean_absolute_error(y_true, y_pred):
    return abs(y_true - y_pred).sum() / len(y_true) #abs() zwraca wartość bezwzględną

mean_absolute_error(y_true, y_pred)

8.013989036607255

Nie muszę za każdym razem pisać tej funkcji, bo jest ona zaimplementowana w bibliotece sklearn.metrics

In [16]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_true, y_pred)

8.013989036607255

Kolejna metryka to #Błąd Średniokwadratowy

### <a name='a4'></a> Mean Squared Error - MSE
### $$MSE = \frac{1}{n}\sum_{i=1}^{n}(y_{true} - y_{pred})^{2}$$

Dzięki podnoszeniu do kwadratu, większe odległości od prawdziwych wyników będą bardziej karane

In [17]:
def mean_squared_error(y_true, y_pred):
    return ((y_true - y_pred) ** 2).sum() / len(y_true)

mean_squared_error(y_true, y_pred)

99.91683775698777

In [19]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_true, y_pred)

99.91683775698777

### <a name='a5'></a> Root Mean Squared Error - RMSE - Pierwiastek błędu średniokwadratowego
### $$RMSE = \sqrt{MSE}$$

Jest to o tyle lepsze, że pozwala nam nieco lepiej zinterpretować nasz wynik.
Pokazuje, jak średnio przewidywania modelu odbiegają od wartości prawdziwych

In [20]:
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(((y_true - y_pred) ** 2).sum() / len(y_true))

root_mean_squared_error(y_true, y_pred)

9.995841022994902

In [21]:
np.sqrt(mean_squared_error(y_true, y_pred))

9.995841022994902

### <a name='a6'></a>  Max Error - Błąd maksymalny

$$ME = max(|y\_true - y\_pred|)$$

In [22]:
def max_error(y_true, y_pred):
    return abs(y_true - y_pred).max()

max_error(y_true, y_pred)

32.1725609742564

In [24]:
from sklearn.metrics import max_error
max_error(y_true, y_pred)

32.1725609742564

### <a name='a7'></a>  R2 score - współczynnik determinacji
### $$R2\_score = 1 - \frac{\sum_{i=1}^{N}(y_{true} - y_{pred})^{2}}{\sum_{i=1}^{N}(y_{true} - \overline{y_{true}})^{2}}$$

To jest ten wykres powyżej sredniej. Zależy nam, żeby metryka ta była jak najbliżej 1

In [25]:
def r2_score(y_true, y_pred):
    numerator = ((y_true - y_pred) ** 2).sum()
    denominator = ((y_true - y_true.mean()) ** 2).sum()
    try:
        r2 = 1 - numerator / denominator
    except ZeroDivisionError:
        print('Dzielenie przez zero')
    return r2

In [26]:
r2_score(y_true, y_pred)

0.7460900082363925

In [27]:
from sklearn.metrics import r2_score
r2_score(y_true, y_pred)

0.7460900082363925